# **Tarea con Polars**

**Primer Ejercicio:**Desarrollo generado a partir de la explicación desarrollada por el profesor Gerardo en la clase teniendo presente las diferentes probabilidades condicionales.

In [ ]:
#Desarrollo de los conjuntos base

conjuntos = [
    {'a','b','c'},
    {'a','b'},
    {'a','c','d'},
    {'a'},
    {'b','c'},
    {'b','c','d'}
]


In [ ]:
import polars as pl
#Conversión a DataFrame de Polars
df = pl.DataFrame({
    "id": range(1, len(conjuntos) + 1),  # Identificador opcional
    "set": conjuntos
})

In [ ]:
import polars as pl

# Function to calculate conditional probability P(A | B)
def probabilidad_condicional(df, A, B):
    total_conjuntos = df.height  # Total of subsets

    # Filter subsets containing B using `map_elements` outside the filter context
    is_subset_B = df["set"].map_elements(lambda s: B.issubset(s))
    subconjuntos_B = df.filter(is_subset_B)

    # Filter subsets containing A and B within the B subset using `apply` outside filter context
    is_subset_A_given_B = subconjuntos_B["set"].map_elements(lambda s: A.issubset(s))
    subconjuntos_AyB = subconjuntos_B.filter(is_subset_A_given_B)

    # Calculate conditional probability
    P_B = len(subconjuntos_B) / total_conjuntos if total_conjuntos > 0 else 0
    P_A_dado_B = len(subconjuntos_AyB) / len(subconjuntos_B) if len(subconjuntos_B) > 0 else 0

    return P_A_dado_B

# Example: P({'c'} | {'a'})
A = {'c'}
B = {'a'}
resultado = probabilidad_condicional(df, A, B)

print(f"P({A} | {B}) =", resultado)


P({'c'} | {'a'}) = 0.5


<ipython-input-14-056bbbf88f1f>:8: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  is_subset_B = df["set"].map_elements(lambda s: B.issubset(s))
<ipython-input-14-056bbbf88f1f>:12: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  is_subset_A_given_B = subconjuntos_B["set"].map_elements(lambda s: A.issubset(s))


**Algoritmo Apriori:** Para el primer ejercicio se tiene:

Para el desarrollo del algoritmo tuve presente lo siguiente:

*   **Conteo Inicial:** Me permitió contar la frecuencia de cada ítem individual.
*   **Generación de Candidatos:** Usa combinación para crear todos los posibles itemsets de tamaño k+1, teniendo presente que mi cojunto de ítems en este caso serían los conjuntos que el profesor colocó en el tablero.
*   **Podado:** Elimina candidatos donde algún subconjunto de tamaño k no sea frecuente.
*   **Cálculo de Soporte:** Cuenta cuántas transacciones contienen cada candidato.
*   **Filtrado:** Conserva solo los conjuntos que cumplen con el soporte mínimo






In [ ]:
from itertools import combinations
from collections import defaultdict
#Desarrollo de diferentes conjuntos de base de datos
conjuntos = [
    {'a','b','c'},
    {'a','b'},
    {'a','c','d'},
    {'a'},
    {'b','c'},
    {'b','c','d'}
]

def apriori(datos, soporte_min):
    # Paso 1: Encontrar itemsets frecuentes de tamaño 1
    conteo = defaultdict(int)
    for transaccion in datos:
        for item in transaccion:
            conteo[frozenset([item])] += 1

    frecuentes = [
        [item for item, count in conteo.items() if count >= soporte_min],
    ]

    k = 1
    while True:
        # Paso 2: Generar candidatos de tamaño k+1
        items = list({item for itemset in frecuentes[k-1] for item in itemset})
        candidatos = [frozenset(comb) for comb in combinations(items, k+1)]
        '''La función frozenset transforma cualquier objeto iterable en un
        objeto inmutable'''

        # Paso 3: Podar candidatos
        candidatos_podados = []
        for candidato in candidatos:
            subconjuntos = [frozenset(s) for s in combinations(candidato, k)]
            if all(sub in frecuentes[k-1] for sub in subconjuntos):
                candidatos_podados.append(candidato)

        # Paso 4: Calcular soporte
        conteo_candidatos = defaultdict(int)
        for transaccion in datos:
            for candidato in candidatos_podados:
                if candidato.issubset(transaccion):
                    conteo_candidatos[candidato] += 1

        # Filtrar por soporte mínimo
        nuevos_frecuentes = [item for item, count in conteo_candidatos.items() if count >= soporte_min]

        if not nuevos_frecuentes:
            break

        frecuentes.append(nuevos_frecuentes)
        k += 1

    return frecuentes

# Ejecutar con soporte mínimo = 2 (33% de los conjuntos)
resultado = apriori(conjuntos, 2)

# Mostrar resultados
for i, nivel in enumerate(resultado):
    print(f"Itemsets frecuentes de tamaño {i+1}:")
    for itemset in nivel:
        print(f"   {set(itemset)}")


Itemsets frecuentes de tamaño 1:
   {'a'}
   {'b'}
   {'c'}
   {'d'}
Itemsets frecuentes de tamaño 2:
   {'a', 'b'}
   {'a', 'c'}
   {'b', 'c'}
   {'c', 'd'}


**Parte de código con probabilidad de confianza:** Teniendo presente que en la clase se expuso todo en términos de probabilidades, se comparte una visión donde se plantea el algoritmo en torno a la visualización de probabilidades.

In [ ]:
from itertools import combinations

# Configuración pensando en términos de probabilidades.
conjuntos = [
    {'a','b','c'},
    {'a','b'},
    {'a','c','d'},
    {'a'},
    {'b','c'},
    {'b','c','d'}
]
min_support = 2
min_confidence = 0.5  # 50%

# ------------------------------------------
# 1. Algoritmo Apriori (Modificado para guardar soportes y tener presente probabilidades)
# ------------------------------------------
def apriori_with_support(datos, soporte_min):
    support_dict = {}
    # Paso 1: Itemsets de tamaño 1
    conteo = defaultdict(int)
    for trans in datos:
        for item in trans:
            key = frozenset([item])
            conteo[key] += 1
    support_dict.update(conteo)
    frecuentes = [[k for k, v in conteo.items() if v >= soporte_min]]

    k = 1
    while True:
        # Generar candidatos
        items = list({i for s in frecuentes[-1] for i in s})
        candidatos = [frozenset(c) for c in combinations(items, k+1)]

        # Podar y calcular soporte
        conteo_candidatos = defaultdict(int)
        for trans in datos:
            for cand in candidatos:
                if cand.issubset(trans):
                    conteo_candidatos[cand] += 1
        support_dict.update(conteo_candidatos)

        nuevos_frec = [k for k, v in conteo_candidatos.items() if v >= soporte_min]
        if not nuevos_frec:
            break
        frecuentes.append(nuevos_frec)
        k += 1

    return support_dict

# Ejecutar Apriori
apriori_support = apriori_with_support(conjuntos, min_support)

# ------------------------------------------
# 3. Función común para generar reglas
# ------------------------------------------
def generar_reglas(support_dict, min_conf):
    reglas = []
    itemsets = [k for k in support_dict.keys() if len(k) > 1]

    for itemset in itemsets:
        items = list(itemset)
        # Generar todos los posibles antecedentes
        for i in range(1, len(items)):
            for antecedente in combinations(items, i):
                antecedente = frozenset(antecedente)
                consecuente = itemset - antecedente
                soporte_total = support_dict[itemset]
                soporte_antecedente = support_dict.get(antecedente, 0)

                if soporte_antecedente == 0:
                    continue

                confianza = soporte_total / soporte_antecedente
                if confianza >= min_conf:
                    reglas.append({
                        'regla': f"{set(antecedente)} => {set(consecuente)}",
                        'soporte': soporte_total,
                        'confianza': round(confianza, 2)
                    })
    return reglas

# Generar reglas para ambos algoritmos
reglas_apriori = generar_reglas(apriori_support, min_confidence)

# 4. Resultados
# ------------------------------------------
print("Reglas de Apriori (Confianza ≥ 50%):")
for regla in reglas_apriori:
    print(f"{regla['regla']} | Soporte: {regla['soporte']}, Confianza: {regla['confianza']}")


Reglas de Apriori (Confianza ≥ 50%):
{'a'} => {'b'} | Soporte: 2, Confianza: 0.5
{'b'} => {'a'} | Soporte: 2, Confianza: 0.5
{'a'} => {'c'} | Soporte: 2, Confianza: 0.5
{'c'} => {'a'} | Soporte: 2, Confianza: 0.5
{'b'} => {'c'} | Soporte: 3, Confianza: 0.75
{'c'} => {'b'} | Soporte: 3, Confianza: 0.75
{'d'} => {'a'} | Soporte: 1, Confianza: 0.5
{'c'} => {'d'} | Soporte: 2, Confianza: 0.5
{'d'} => {'c'} | Soporte: 2, Confianza: 1.0
{'d'} => {'b'} | Soporte: 1, Confianza: 0.5
{'a', 'b'} => {'c'} | Soporte: 1, Confianza: 0.5
{'a', 'c'} => {'b'} | Soporte: 1, Confianza: 0.5
{'d'} => {'a', 'c'} | Soporte: 1, Confianza: 0.5
{'a', 'c'} => {'d'} | Soporte: 1, Confianza: 0.5
{'a', 'd'} => {'c'} | Soporte: 1, Confianza: 1.0
{'c', 'd'} => {'a'} | Soporte: 1, Confianza: 0.5
{'d'} => {'b', 'c'} | Soporte: 1, Confianza: 0.5
{'b', 'd'} => {'c'} | Soporte: 1, Confianza: 1.0
{'c', 'd'} => {'b'} | Soporte: 1, Confianza: 0.5


**Algoritmo FP-Growth:**Para el primer ejercicio se tiene lo siguiente:

In [ ]:
from collections import defaultdict

conjuntos = [
    {'a','b','c'},
    {'a','b'},
    {'a','c','d'},
    {'a'},
    {'b','c'},
    {'b','c','d'}
]

class TreeNode:
    def __init__(self, name, parent):
        self.name = name       # Nombre del ítem
        self.parent = parent   # Nodo padre
        self.children = {}     # Diccionario de hijos {nombre: nodo}
        self.count = 0         # Contador de soporte
        self.link = None       # Enlace a nodos del mismo ítem

def build_fp_tree(conjuntos, min_support):
    # Paso 1: Calcular frecuencia de ítems
    item_counts = defaultdict(int)
    for trans in conjuntos:
        for item in trans:
            item_counts[item] += 1

    # Filtrar ítems frecuentes y ordenar
    freq_items = {item: count for item, count in item_counts.items() if count >= min_support}
    if not freq_items:
        return None, None
    sorted_items = sorted(freq_items.items(), key=lambda x: (-x[1], x[0]))
    order = [item[0] for item in sorted_items]

    # Paso 2: Construir FP-Tree y tabla de cabecera
    root = TreeNode('root', None)
    header_table = {item: [count, None] for item, count in sorted_items}

    for trans in conjuntos:
        # Filtrar y ordenar transacción
        filtered = [item for item in trans if item in freq_items]
        filtered.sort(key=lambda x: (order.index(x), x))
        if not filtered:
            continue

        # Insertar en el árbol
        current_node = root
        for item in filtered:
            if item in current_node.children:
                current_node = current_node.children[item]
                current_node.count += 1
            else:
                new_node = TreeNode(item, current_node)
                new_node.count = 1
                current_node.children[item] = new_node

                # Enlazar a la tabla de cabecera
                if header_table[item][1] is None:
                    header_table[item][1] = new_node
                else:
                    node = header_table[item][1]
                    while node.link is not None:
                        node = node.link
                    node.link = new_node
                current_node = new_node

    return root, header_table

def mine_fp_tree(header_table, min_support, prefix, freq_itemsets):
    items = list(header_table.items())
    # Procesar desde el ítem menos frecuente
    for item, (count, node) in sorted(items, key=lambda x: (x[1][0], x[0])):
        new_prefix = prefix.copy()
        new_prefix.add(item)
        freq_itemsets.append((new_prefix, count))  # Añadir itemset actual

        # Paso 3: Encontrar patrones condicionales
        conditional_paths = []
        current_node = node
        while current_node is not None:
            path = []
            parent = current_node.parent
            while parent.name != 'root':
                path.append(parent.name)
                parent = parent.parent
            path.reverse()
            if path:
                conditional_paths.append((path, current_node.count))
            current_node = current_node.link

        # Construir transacciones condicionales
        conditional_trans = []
        for path, cnt in conditional_paths:
            conditional_trans.extend([path] * cnt)

        # Construir sub-árbol condicional
        sub_root, sub_header = build_fp_tree(conditional_trans, min_support)
        if sub_header is not None:
            mine_fp_tree(sub_header, min_support, new_prefix, freq_itemsets)

def fp_growth(transacciones, min_support):
    freq_itemsets = []
    root, header_table = build_fp_tree(transacciones, min_support)
    if header_table:
        mine_fp_tree(header_table, min_support, set(), freq_itemsets)
    # Filtrar y formatear resultados
    result = defaultdict(int)
    for itemset, count in freq_itemsets:
        key = frozenset(itemset)
        result[key] = max(result[key], count)
    return sorted([(set(k), v) for k, v in result.items()], key=lambda x: (len(x[0]), x[0]))

# Ejecutar con soporte mínimo = 2
resultado = fp_growth(conjuntos, 2)

# Mostrar resultados Básicos de la configuración de los diferentes ítems
print("Itemsets frecuentes con FP-Growth:")
for itemset, soporte in resultado:
    print(f"{set(itemset)}: Soporte = {soporte}")



# ------------------------------------------
# 3. Función común para generar reglas
# ------------------------------------------
def generar_reglas(support_dict, min_conf):
    reglas = []
    itemsets = [k for k in support_dict.keys() if len(k) > 1]

    for itemset in itemsets:
        items = list(itemset)
        # Generar todos los posibles antecedentes
        for i in range(1, len(items)):
            for antecedente in combinations(items, i):
                antecedente = frozenset(antecedente)
                consecuente = itemset - antecedente
                soporte_total = support_dict[itemset]
                soporte_antecedente = support_dict.get(antecedente, 0)

                if soporte_antecedente == 0:
                    continue

                confianza = soporte_total / soporte_antecedente
                if confianza >= min_conf:
                    reglas.append({
                        'regla': f"{set(antecedente)} => {set(consecuente)}",
                        'soporte': soporte_total,
                        'confianza': round(confianza, 2)
                    })
    return reglas

# Generar reglas para ambos algoritmos
reglas_fp_growth = generar_reglas(fp_growth_support, min_confidence)

print("\nReglas de FP-Growth (Confianza ≥ 50%):")
for regla in reglas_fp_growth:
    print(f"{regla['regla']} | Soporte: {regla['soporte']}, Confianza: {regla['confianza']}")

Itemsets frecuentes con FP-Growth:
{'d'}: Soporte = 2
{'a'}: Soporte = 4
{'b'}: Soporte = 4
{'c'}: Soporte = 4
{'c', 'd'}: Soporte = 2
{'a', 'b'}: Soporte = 2
{'a', 'c'}: Soporte = 2
{'c', 'b'}: Soporte = 3

Reglas de FP-Growth (Confianza ≥ 50%):
{'c'} => {'d'} | Soporte: 2, Confianza: 0.5
{'d'} => {'c'} | Soporte: 2, Confianza: 1.0
{'a'} => {'b'} | Soporte: 2, Confianza: 0.5
{'b'} => {'a'} | Soporte: 2, Confianza: 0.5
{'a'} => {'c'} | Soporte: 2, Confianza: 0.5
{'c'} => {'a'} | Soporte: 2, Confianza: 0.5
{'c'} => {'b'} | Soporte: 3, Confianza: 0.75
{'b'} => {'c'} | Soporte: 3, Confianza: 0.75
